In [ ]:
# Copyright (c) 2020 ZZH

In [ ]:
# simplified : replace the stem structure with one convolutional layer, channels are divided by 4

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, MaxPool2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras import Model
import os
import numpy as np

In [ ]:
gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
epochs = 100
lr = 0.1
batch_size = 128
REGULARIZER  = 0.0005
checkpoint_save_path =  './Model/InceptionV1/'
log_dir = os.path.join("Model","InceptionV1_logs")

In [ ]:
#数据导入及数据增强
cifar10 = tf.keras.datasets.cifar10
(x_train,y_train),(x_test,y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
mean = [125.307, 122.95, 113.865]  #np.mean()
std = [62.9932, 62.0887, 66.7048]  #np.std()
for i in range(3):
    x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
    x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]
DataGenTrain = tf.keras.preprocessing.image.ImageDataGenerator(
               rotation_range = 15,
               width_shift_range = 0.1,
               height_shift_range = 0.1,
               horizontal_flip = True,
               vertical_flip = False,
               shear_range=0.1,
               zoom_range = 0.1)
DataGenTrain.fit(x_train)

In [ ]:
def scheduler(epoch):
    if epoch < 40:
        return 0.1
    if epoch < 60:
        return 0.05
    if epoch < 85:    
        return 0.01
    return 0.002

In [ ]:
class ConvBNRelu(Model):
    def __init__(self,channels,kernel_size,strides,padding):
        super(ConvBNRelu,self).__init__()
        self.model = tf.keras.models.Sequential([
                    Conv2D(filters=channels, kernel_size=kernel_size,strides=strides, padding=padding,
                           kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER)),
                    BatchNormalization(momentum=0.9),
                    Activation('relu') ])
    def call(self,inputs):
        outputs = self.model(inputs)
        return outputs

In [ ]:
class InceptionBlk(Model):
    def __init__(self,channels_1,channels_2_1,channels_2_2,channels_3_1,channels_3_2,channels_4,strides):
        super(InceptionBlk,self).__init__()            
        self.c1 = ConvBNRelu(channels=channels_1,kernel_size=1,strides=strides, padding='same')
        self.c2_1 = ConvBNRelu(channels=channels_2_1,kernel_size=1,strides=strides, padding='same')
        self.c2_2 = ConvBNRelu(channels=channels_2_2,kernel_size=3,strides=1, padding='same')
        self.c3_1 = ConvBNRelu(channels=channels_3_1,kernel_size=1,strides=strides, padding='same')
        self.c3_2 = ConvBNRelu(channels=channels_3_2,kernel_size=5,strides=1, padding='same')
        self.p4_1 = MaxPool2D(pool_size=3,strides=1, padding='same')
        self.c4_2 = ConvBNRelu(channels=channels_4,kernel_size=1,strides=strides, padding='same')
    def call(self,x):
        x1 = self.c1(x)
        x2_1 = self.c2_1(x)
        x2_2 = self.c2_2(x2_1)
        x3_1 = self.c3_1(x)
        x3_2 = self.c3_2(x3_1)
        x4_1 = self.p4_1(x)
        x4_2 = self.c4_2(x4_1)
        x = tf.concat([x1,x2_2,x3_2,x4_2], axis = -1)
        return x

In [ ]:
#网络搭建及训练
class InceptionV1(Model):
    def __init__(self):
        super(InceptionV1,self).__init__()
        self.c1 = ConvBNRelu(channels=32,kernel_size=3,strides=1, padding='same')
        self.blocks = tf.keras.models.Sequential()
        self.blocks.add(InceptionBlk(channels_1=16,channels_2_1=24,channels_2_2=32,channels_3_1=4,channels_3_2=8,channels_4=8,strides=1))
        self.blocks.add(InceptionBlk(channels_1=32,channels_2_1=32,channels_2_2=48,channels_3_1=8,channels_3_2=16,channels_4=16,strides=1))
        self.blocks.add(MaxPool2D(pool_size=3,strides=2, padding='same'))
        self.blocks.add(InceptionBlk(channels_1=48,channels_2_1=24,channels_2_2=52,channels_3_1=4,channels_3_2=12,channels_4=16,strides=1))
        self.blocks.add(InceptionBlk(channels_1=40,channels_2_1=28,channels_2_2=56,channels_3_1=6,channels_3_2=16,channels_4=16,strides=1))
        self.blocks.add(InceptionBlk(channels_1=32,channels_2_1=32,channels_2_2=64,channels_3_1=6,channels_3_2=16,channels_4=16,strides=1))
        self.blocks.add(InceptionBlk(channels_1=28,channels_2_1=36,channels_2_2=72,channels_3_1=8,channels_3_2=16,channels_4=16,strides=1))
        self.blocks.add(InceptionBlk(channels_1=64,channels_2_1=40,channels_2_2=80,channels_3_1=8,channels_3_2=32,channels_4=32,strides=1))
        self.blocks.add(MaxPool2D(pool_size=3,strides=2, padding='same'))
        self.blocks.add(InceptionBlk(channels_1=64,channels_2_1=40,channels_2_2=80,channels_3_1=8,channels_3_2=32,channels_4=32,strides=1))
        self.blocks.add(InceptionBlk(channels_1=96,channels_2_1=48,channels_2_2=96,channels_3_1=12,channels_3_2=32,channels_4=32,strides=1))
        self.p1 = GlobalAveragePooling2D()
        self.d1 = Dropout(0.4)
        self.f1 = Dense(10,activation='softmax',kernel_regularizer=tf.keras.regularizers.l2(REGULARIZER))  
    def call(self,x):
        x = self.c1(x)
        x = self.blocks(x)
        x = self.p1(x)
        x = self.d1(x)
        y = self.f1(x)
        return y

In [ ]:
model = InceptionV1()

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True),
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

callbacks = [
            tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.ModelCheckpoint(     #模型保存
                filepath = checkpoint_save_path,
                save_weights_only = False,
                monitor = 'val_accuracy',
                save_best_only = True),
#             tf.keras.callbacks.EarlyStopping(       #早停
#                 monitor = 'val_accuracy',
#                 patience=15, 
#                 baseline=None),
            tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True, write_images=False)  #保存计算图
]

hist = model.fit(DataGenTrain.flow(x_train,y_train,batch_size=batch_size,shuffle=True),
                 epochs=epochs,
                 validation_data=(x_test,y_test),
                 validation_freq=1,
                 callbacks=callbacks)

model.summary()

In [ ]:
#结果可视化
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
plt.style.use({'figure.figsize':(6,4)})

plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()
plt.show()

In [ ]:
print('best result: {:.2f}%  ({}epochs)'.format(100*max(hist.history['val_accuracy']),1+hist.history['val_accuracy'].index(max(hist.history['val_accuracy']))))
# best result: 93.02%  (93epoch)

In [ ]:
#tensorboard可视化
#!tensorboard --logdir=./Model/InceptionV1_logs
#http://localhost:6006/